In [1]:
import pandas as pd
import os
import numpy as np
from tqdm import tqdm
from PIL import Image

import torch
from torchvision.models import resnet50, ResNet50_Weights
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split
from torchsummary import summary
from datasets import load_metric

In [2]:
#путь к файлам
path_to_images = 'all_images'

train_csv = pd.read_csv('files_train.csv')
eval_csv = pd.read_csv('files_eval.csv')
test_csv = pd.read_csv('files_test.csv')

In [10]:
transform = transforms.Compose([
    torchvision.transforms.Resize(
        size=(224, 224)
    ),
    transforms.ToTensor()
])

In [11]:
class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, images_csv, dir_files, transform=None):
        self.dir_files = dir_files
        self.label = images_csv 
        self.transform = transform
        self.file_arr = np.array(self.label['files'].to_list())
        self.label_arr = np.array(self.label['classes'].to_list())
    def __len__(self):
        return len(self.label)
    
    def __getitem__(self, idx):
        file_name = os.path.join(self.dir_files, str(self.file_arr[idx]))
        image = Image.open(file_name)
        image = image.convert('RGB')
        label = self.label_arr[idx]
        if self.transform:
            feature = self.transform(image)
        sample = {
            'pixel_values': feature, 
            'label': int(label),
            'file_name': file_name
        }
        return sample 

In [12]:
train_set = ImageDataset(images_csv = train_csv, dir_files = path_to_images, transform = transform)
eval_set = ImageDataset(images_csv = eval_csv, dir_files = path_to_images, transform = transform)
test_set = ImageDataset(images_csv = test_csv, dir_files = path_to_images, transform = transform)

In [13]:
print(len(train_set))
print(len(eval_set))
print(len(test_set))

2100
300
600


In [22]:
batch_size = 50

In [23]:
loader_train = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
loader_eval = torch.utils.data.DataLoader(eval_set, batch_size=batch_size, shuffle=True)
loader_test = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True)

In [24]:
model = torchvision.models.resnet50(pretrained=False,progress = True)

In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [26]:
device

device(type='cuda')

In [27]:
model = model.to(device)

In [28]:
num_epochs = 150
learning_rate = 0.001
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [29]:
for epoch in range(num_epochs):
    model.train()
    for batch in tqdm(loader_train):
        # Move input and label tensors to the device
        inputs = batch["pixel_values"].to(device)
        labels = batch["label"].to(device)

        # Zero out the optimizer
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()
        optimizer.step()
        torch.cuda.empty_cache()

    # Print the loss for every epoch
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')
  
    
    accuracy = load_metric("accuracy")
    model.eval()
    for batch_1 in tqdm(loader_eval):
        inputs = batch_1["pixel_values"].to(device)
        labels = batch_1["label"].to(device)
        
        outputs = model(inputs)
        predictions = outputs.argmax(-1).cpu().detach().numpy()
        _, predictions = torch.max(outputs.data, 1)
        references = batch_1["label"].numpy()
        accuracy.add_batch(predictions=predictions, references=references)
    final_score = accuracy.compute()
    print("Accuracy on eval set:", final_score)

print(f'Finished Training, Loss: {loss.item():.4f}')

100%|██████████████████████████████████████████████████████████████████████████████████| 42/42 [00:24<00:00,  1.74it/s]
C:\Users\Елизавета\AppData\Local\Temp\ipykernel_1520\676513376.py:24: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  accuracy = load_metric("accuracy")


Epoch 1/150, Loss: 0.9172


100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:02<00:00,  2.55it/s]


Accuracy on eval set: {'accuracy': 0.48333333333333334}


100%|██████████████████████████████████████████████████████████████████████████████████| 42/42 [00:23<00:00,  1.82it/s]


...

Accuracy on eval set: {'accuracy': 0.7233333333333334}


100%|██████████████████████████████████████████████████████████████████████████████████| 42/42 [00:22<00:00,  1.87it/s]


Epoch 150/150, Loss: 0.0000


100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  3.29it/s]

Accuracy on eval set: {'accuracy': 0.74}
Finished Training, Loss: 0.0000
